In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import os
import shutil
import numpy as np

from sklearn.utils import shuffle

from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [4]:
train_dir = r'C:\Users\subrahm\Documents\breast-cancer-detection\Breast-Cancer-Detection\data\test_cnn_data'
dest_dir = r'C:\Users\subrahm\Documents\breast-cancer-detection\Breast-Cancer-Detection\data\train_data\test_cnn\all_images'
counter = 0

In [5]:
if len(os.listdir(dest_dir)) == 0:
    for subdir, dirs, files in os.walk(train_dir):
        for file in files:
            full_path = os.path.join(subdir, file)
            shutil.copy(full_path, dest_dir)
            counter = counter + 1

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\subrahm\\Documents\\breast-cancer-detection\\Breast-Cancer-Detection\\data\\train_data\\test_cnn\\all_images'

In [ ]:
len_files = len(os.listdir(dest_dir))
filenames_counter = 0
labels_counter = -1

In [ ]:
filenames = []
labels = np.zeros((len_files, 1))

In [ ]:
for subdir, dirs, files in os.walk(train_dir):
    for file in files:
        filenames.append(file)
        labels[filenames_counter, 0] = labels_counter
        filenames_counter = filenames_counter + 1
    labels_counter = labels_counter+1
    
print(len(filenames))
print(labels.shape)

In [9]:
# saving the filename array as .npy file
np.save('../../data/train_data/test_cnn/filenames.npy', filenames)

In [10]:
# One hot vector representation of labels
y_labels_one_hot = to_categorical(labels)

In [11]:
# saving the y_labels_one_hot array as a .npy file
np.save('../../data/train_data/test_cnn/y_labels_one_hot.npy', y_labels_one_hot)

In [12]:
filenames_shuffled, y_labels_one_hot_shuffled = shuffle(filenames, y_labels_one_hot)

# saving the shuffled file.
# you can load them later using np.load().
np.save('../../data/train_data/test_cnn/y_labels_one_hot_shuffled.npy', y_labels_one_hot_shuffled)
np.save('../../data/train_data/test_cnn/filenames_shuffled.npy', filenames_shuffled)

In [13]:
# Used this line as our filename array is not a numpy array.
filenames_shuffled_numpy = np.array(filenames_shuffled)

In [14]:
X_train_filenames, X_val_filenames, y_train, y_val = train_test_split(
    filenames_shuffled_numpy, y_labels_one_hot_shuffled, test_size=0.2, random_state=1)

In [15]:
print(X_train_filenames.shape) # (3644,)
print(y_train.shape)           # (3644, 2)

print(X_val_filenames.shape)   # (911,)
print(y_val.shape)             # (911, 2)


(43,)
(43, 2)
(11,)
(11, 2)


In [16]:
# You can save these files as well. As you will be using them later for training and validation of your model.
np.save('../../data/train_data/test_cnn/X_train_filenames.npy', X_train_filenames)
np.save('../../data/train_data/test_cnn/y_train.npy', y_train)

np.save('../../data/train_data/test_cnn/X_val_filenames.npy', X_val_filenames)
np.save('../../data/train_data/test_cnn/y_val.npy', y_val)

In [17]:
shutil.make_archive(dest_dir, "zip", dest_dir)

'C:\\Users\\subrahm\\Documents\\breast-cancer-detection\\Breast-Cancer-Detection\\data\\train_data\\test_cnn\\all_images.zip'

In [16]:
class My_Custom_Generator(keras.utils.Sequence) :
  
    def __init__(self, image_filenames, labels, batch_size) :
        self.image_filenames = image_filenames
        self.labels = labels
        self.batch_size = batch_size

    
    def __len__(self) :
        return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)
  
  
    def __getitem__(self, idx) :
        batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]

        return np.array([
                resize(imread(os.path.join(dest_dir + str(file_name))), (80, 80, 3))
                   for file_name in batch_x])/255.0, np.array(batch_y)

In [ ]:
batch_size = 32

my_training_batch_generator = My_Custom_Generator(X_train_filenames, y_train, batch_size)
my_validation_batch_generator = My_Custom_Generator(X_val_filenames, y_val, batch_size)